# **Introduction**

> The purpose of this notebook is to analyze different attributes affecting the passResult, specifically when the play results in a "Sack". The reason to choose this attribute as the prediction class is that it has a very high impact in the game outcome. For this analysis i have chosen one team's defence against all other teams offence. The team i choose is "philadelphia eagles".To create a consensus i have created a classification model which includes the most important variables affecting the predictor variable "passResult".
> 


# Features:
> 1. NumberofPassRushers
> 2. DefendersInTheBox
> 3. TypeOfDropBack
> 4. possessionTeam
> 5. personnelD
> 6. Quarter
> 7. Downs

# Predictor:
> PassResult ( Sack | No Sack)

**Libraries:**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from scipy.stats import chisquare as testt, chi2_contingency
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score

# Import Data:

> 1. We are analyzing on team 'PHI', so the data includes all weeks data that includes philidelphia matches played in the following weeks ( week12,Week5, Week7, Week3, Week10, Week13, Week16).

> Additionally we are importing the plays, players and games information.



In [ ]:
games = pd.read_csv("../input/nfl-big-data-bowl-2021/games.csv")
players = pd.read_csv("../input/nfl-big-data-bowl-2021/players.csv")
plays = pd.read_csv("../input/nfl-big-data-bowl-2021/plays.csv")
week1 = pd.read_csv("../input/nfl-big-data-bowl-2021/week12.csv")
week11 = pd.read_csv("../input/nfl-big-data-bowl-2021/week1.csv")
week3 = pd.read_csv("../input/nfl-big-data-bowl-2021/week3.csv")
week5 = pd.read_csv("../input/nfl-big-data-bowl-2021/week5.csv")
week7 = pd.read_csv("../input/nfl-big-data-bowl-2021/week7.csv")
week10 = pd.read_csv("../input/nfl-big-data-bowl-2021/week10.csv")
week13 = pd.read_csv("../input/nfl-big-data-bowl-2021/week13.csv")
week16 = pd.read_csv("../input/nfl-big-data-bowl-2021/week16.csv")

week2 = week1.loc[(week1["gameId"]==2018112506),]
week2 = week2.reset_index()

week21 = week11.loc[(week11["gameId"]==2018090600) ,]
week21 = week21.reset_index()

week31 = week3.loc[(week3["gameId"]==2018092308) ,]
week31 = week31.reset_index()

week51 = week5.loc[(week5["gameId"]==2018100709) ,]
week51 = week51.reset_index()

week71 = week7.loc[(week7["gameId"]==2018102108) ,]
week71 = week71.reset_index()

week101 = week10.loc[(week10["gameId"]==2018111111) ,]
week101 = week101.reset_index()

week131 = week13.loc[(week13["gameId"]==2018120300) ,]
week131 = week131.reset_index()

week161 = week16.loc[(week16["gameId"]==2018122310),]
week161 = week161.reset_index()
week1.append(week21)
week1.append(week31)
week1.append(week51)
week1.append(week71)
week1.append(week101)
week1.append(week131)
week1.append(week161)

**Target :**

> Next, since we are analyzing the defence performace of philadelphia eagles i have combined incomplete , intercepted and Complete passResult into (NO SACK) 'NS', and created a binary class having 'S' and 'NS' categories.

In [ ]:
plays.loc[plays["passResult"].isin(['I','IN','C']),"passResult"] = 'NS'

plays3 = plays[plays["passResult"].isin(['NS','S'])]
plays2 = plays3.merge(week1,how='inner',on=['gameId', 'playId'])

# Data :

In [ ]:


print(plays2.head())

> Below id Chisqaure test i conducted to find id there exists a relationship between personallD and passresult.

In [ ]:
sample1 = pd.read_csv("../input/test-sample/chisquare.csv")

sample1 = sample1.set_index("personnelD")

print(sample1.head())



result = testt(f_obs=sample1)

pvalue = chi2_contingency(sample1)[1]

print("Pvalue :" ,int(pvalue))


The Pvalue indicates we can reject the null hypothesis and accept the hypothesis that there indeed exists a relationship between the attributes.

Next, i used a label encoder to impute the class variable.

# Code:


In [ ]:
encoder = LabelEncoder()
plays2["passResult_1"] = encoder.fit_transform(plays2["passResult"])
plays2 = plays2[plays2["possessionTeam"] != 'PHI']

df = plays2[["passResult_1","numberOfPassRushers","defendersInTheBox","typeDropback","personnelD","possessionTeam","quarter","down"]]
df = df[df["passResult_1"].notna()]
df = df[df["numberOfPassRushers"].notna()]
df = df[df["typeDropback"].notna()]


df = pd.get_dummies(df, prefix=["Def"],columns=["defendersInTheBox"])
df = pd.get_dummies(df, prefix=["Pass"],columns=["numberOfPassRushers"])
df = pd.get_dummies(df, prefix=["Form"],columns=["personnelD"])
df = pd.get_dummies(df,prefix=["Team"],columns=["possessionTeam"])
df = pd.get_dummies(df,prefix=["QB"],columns=["typeDropback"])
df = pd.get_dummies(df,prefix=["Q"],columns=["quarter"])
df = pd.get_dummies(df,prefix=["Down"],columns=["down"])

X =df.iloc[:, 1:]
y =df.iloc[:,0]


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.30, random_state=8)


clf = DecisionTreeClassifier()

model = clf.fit(X_train,y_train)

pred = model.predict(X_test)



# Evaluation :
> Evaluation i have used a confusionMatrix and Kappa score to understand the result better.

In [ ]:

print("SCORE :",cohen_kappa_score(y_test,pred))

tp, fp ,  fn ,tn  = confusion_matrix(y_test,pred).ravel()
print('% Correct Prediction : SACK', (tn/(tn + fn)*100))

df5 = pd.DataFrame()

df5["columns"] = X_train.columns
df5["importance"] = clf.feature_importances_

df5 = df5[df5["importance"] > 0.01]

df5 = df5.sort_values(ascending=False,by="importance")

sns.barplot(data = df5, y="columns",x="importance")
plt.show()

# Outcomes:


> *The model was easily able to well approximate the distinction between sacks and non sacks based on certain features.*

**Player_Movement:**
 QB DropBack - Based on given data it is observed that "SCRAMBLE" dropback he is more susceptible to getting sacked.

**Play Time**:
The third and the fourth quarter of the game seems to conceed more sacks of the QB, which may be a good indication of the player's mindset or the team strategies at the end of the play. This can be further explored by the teams.

**PassRushers:** The model explains that in the past when the philadelphia eagles employed 4,5 passRushers they were able to contribute to better to generate a positive outcome for the team. 

**Defenders:** A combination of 6 defenders has worked well with the team in generating a friendly result for the team.

**Formation:** The model states that certain formations such as "4 Defend lineman 2 line backers 5 Defence backs created more pressure to the opposition.

**Plays:**
As per the past games of philadelphia eagles it seems that certain plays significantly contributed to the passResult mostly 2nd Down which may be an indication that the defending created enough pressure in the intial down to make the offence blunder in a QB Sack.

**Team:** The model does showcase that philadelphia eagles perform better against certain teams.

# Future Directions:

> 1. This is an inital analysis which can be greatly explored further.
> 1. This one team model can be used as a base model to understand team plays of all teams.
> 1. Further, the conclusions from the analysis can be assesed for greater clarity.
> 1. Different  tests can be applied to come to test the outcome hypothesis.
> 1. New attributes can be explored to create a more robust model.


